In [ ]:
!pip install pretty_midi networkx numpy

In [ ]:
import pretty_midi
import numpy as np
import networkx as nx
from collections import defaultdict, Counter
from math import log

midi_path = "beethoven_fr_elise_piano_version.mid"
pm = pretty_midi.PrettyMIDI(midi_path)

# Collect (start_time, pitch) for all NON-drum instruments
events = []
for inst in pm.instruments:
    if inst.is_drum:
        continue
    for note in inst.notes:
        events.append((note.start, note.pitch))

# Sort by time
events.sort(key=lambda x: x[0])

# Group by onset (within small tolerance to handle float noise)
TIME_TOL = 1e-3
time_slices = []  # list of sets of pitches
current_time = None
current_chord = []

for t, p in events:
    if current_time is None:
        current_time = t
        current_chord = [p]
    elif abs(t - current_time) <= TIME_TOL:
        current_chord.append(p)
    else:
        time_slices.append(sorted(set(current_chord)))
        current_time = t
        current_chord = [p]

if current_chord:
    time_slices.append(sorted(set(current_chord)))


In [ ]:
# Build directed weighted edges
edge_weights = Counter()
nodes = set()

for chord_a, chord_b in zip(time_slices[:-1], time_slices[1:]):
    for i in chord_a:
        for j in chord_b:
            if i == j:
                continue  # remove self-loops right away
            edge_weights[(i, j)] += 1
            nodes.add(i)
            nodes.add(j)

# Create a NetworkX DiGraph
G = nx.DiGraph()
for (u, v), w in edge_weights.items():
    G.add_edge(u, v, weight=w)

# In case any isolated nodes appear (unlikely here)
for n in nodes:
    if n not in G:
        G.add_node(n)


In [ ]:
n = G.number_of_nodes()
m = G.number_of_edges()
density = nx.density(G)

print("Vertices (notes):", n)
print("Edges:", m)
print("Density:", density)


In [ ]:
def weighted_reciprocity(G):
    W = 0.0
    W_bidir = 0.0
    for u, v, data in G.edges(data=True):
        w_uv = data["weight"]
        W += w_uv
        if G.has_edge(v, u):
            w_vu = G[v][u]["weight"]
            W_bidir += min(w_uv, w_vu)
    if W == 0:
        return 0.0
    return W_bidir / W

r_real = weighted_reciprocity(G)

import random

def shuffle_outgoing_weights_preserve_strength(G):
    H = G.copy()
    for u in H.nodes():
        out_edges = list(H.out_edges(u, data=True))
        if len(out_edges) <= 1:
            continue
        weights = [d["weight"] for _, _, d in out_edges]
        random.shuffle(weights)
        for (idx, (src, dst, data)) in enumerate(out_edges):
            data["weight"] = weights[idx]
    return H

H_null = shuffle_outgoing_weights_preserve_strength(G)
r_null = weighted_reciprocity(H_null)

rho_w = (r_real - r_null) / (1 - r_null) if r_null != 1 else 0.0

print("Weighted reciprocity r_real:", r_real)
print("Weighted reciprocity r_null:", r_null)
print("Weighted reciprocity rho_w:", rho_w)


In [ ]:
def node_entropy(G, u):
    out_edges = list(G.out_edges(u, data=True))
    k = len(out_edges)
    if k <= 1:
        return 0.0
    weights = np.array([d["weight"] for _, _, d in out_edges], dtype=float)
    p = weights / weights.sum()
    H = -np.sum(p * np.log(p))
    H_max = np.log(k)
    return H / H_max if H_max > 0 else 0.0

entropies = [node_entropy(G, u) for u in G.nodes()]
mean_entropy = np.mean(entropies)

print("Mean node entropy:", mean_entropy)


In [ ]:
def global_efficiency_unweighted(G):
    # Shortest path lengths by hop count
    sp = dict(nx.all_pairs_shortest_path_length(G))
    nodes = list(G.nodes())
    n = len(nodes)
    s = 0.0
    count = 0
    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            d = sp.get(i, {}).get(j, np.inf)
            if np.isfinite(d) and d > 0:
                s += 1.0 / d
                count += 1
    return s / count if count > 0 else 0.0

def global_efficiency_weighted(G):
    H = G.copy()
    for u, v, data in H.edges(data=True):
        # cost = 1 / w, and we only assumed w>=1
        data["cost"] = 1.0 / data["weight"]
    # Use Dijkstra on 'cost'
    sp = dict(nx.all_pairs_dijkstra_path_length(H, weight="cost"))
    nodes = list(H.nodes())
    n = len(nodes)
    s = 0.0
    count = 0
    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            d = sp.get(i, {}).get(j, np.inf)
            if np.isfinite(d) and d > 0:
                s += 1.0 / d
                count += 1
    return s / count if count > 0 else 0.0

E_unweighted = global_efficiency_unweighted(G)
E_weighted   = global_efficiency_weighted(G)

print("Unweighted efficiency:", E_unweighted)
print("Weighted efficiency:", E_weighted)


In [ ]:
interval_counts = np.zeros(12, dtype=float)

for u, v, data in G.edges(data=True):
    # Count each transition as many times as its weight
    w = data["weight"]
    interval = (v - u) % 12
    interval_counts[interval] += w

# L2 normalize
norm = np.linalg.norm(interval_counts)
if norm > 0:
    interval_vec = interval_counts / norm
else:
    interval_vec = interval_counts

interval_names = [
    "unison", "m2", "M2", "m3", "M3", "P4", "TT", "P5",
    "m6", "M6", "m7", "M7"
]

print("Interval embedding (L2 normalized):")
for name, val in zip(interval_names, interval_vec):
    print(f"{name}: {val:.4f}")
